# Chunking Strategien - Einfache Erklärung

Dieses Notebook zeigt die drei verschiedenen Chunking-Strategien, die in unserem RAG-System verwendet werden:

1. **Rekursives Chunking**: Teilt Text nach Zeichenanzahl mit Überlappung
2. **Embedding-basiertes semantisches Chunking**: Teilt Text basierend auf semantischer Ähnlichkeit zwischen Satz-Embeddings
3. **LLM-basiertes semantisches Chunking**: Nutzt ein LLM, um sinnvolle Abschnitte zu identifizieren

Wir verwenden einen einfachen Beispieltext über Künstliche Intelligenz.

In [ ]:
beispiel_text = """
Künstliche Intelligenz (KI) ist ein Teilgebiet der Informatik. Sie befasst sich mit der Automatisierung intelligenten Verhaltens.
Machine Learning ist eine wichtige Methode der KI. Dabei lernen Computer aus Daten, ohne explizit programmiert zu werden.
Es gibt verschiedene Arten: überwachtes Lernen, unüberwachtes Lernen und verstärkendes Lernen. Deep Learning ist eine spezielle Form des Machine Learning.
Es verwendet neuronale Netze mit vielen Schichten.
Diese Netze können komplexe Muster in großen Datenmengen erkennen. Besonders erfolgreich ist Deep Learning in der Bildverarbeitung und Spracherkennung. Large Language Models (LLMs) sind eine neuere Entwicklung.
Sie basieren auf der Transformer-Architektur.
Modelle wie GPT können Text generieren, übersetzen und Fragen beantworten. Sie werden mit riesigen Textmengen trainiert.
Die Anwendungen von KI sind vielfältig.
In der Medizin hilft KI bei der Diagnose von Krankheiten. In der Industrie optimiert sie Produktionsprozesse. Im Alltag nutzen wir KI in Sprachassistenten und Empfehlungssystemen.
"""

## 2. Embedding-basiertes semantisches Chunking

**Was ist embedding-basiertes semantisches Chunking?**

Diese Methode teilt Text basierend auf **semantischer Ähnlichkeit** statt auf Zeichenanzahl. Der Algorithmus:

1. Berechnet **Embeddings** für jeden Satz
2. Vergleicht die **Ähnlichkeit** aufeinanderfolgender Sätze
3. Erstellt eine **Grenze** (neuer Chunk), wenn die Ähnlichkeit stark sinkt

**Wichtige Parameter:**

- `breakpoint_threshold_type`: Wie wird entschieden, wann ein neuer Chunk beginnt?
  - **"percentile"**: Nutzt Perzentile der Ähnlichkeiten (z.B. 50 = Median)
  - **"standard_deviation"**: Nutzt Standardabweichung (z.B. 1.5σ)
  - **"interquartile"**: Nutzt Interquartilsabstand (robuster gegen Ausreißer)
  
- `breakpoint_threshold_amount`: Der Schwellenwert
  - Bei "percentile": 50 = Median, 75 = oberes Quartil
  - Bei "standard_deviation": 1.0 = 1σ, 2.0 = 2σ
  - Höhere Werte = weniger Chunks (nur bei sehr großen Unterschieden trennen)

In [ ]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai import OpenAIEmbeddings
import os

def test_semantic_chunking(threshold_type="percentile", threshold_amount=50):
    embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
    # Erstelle Semantic Chunker
    semantic_splitter = None

    semantic_chunks = semantic_splitter.split_text(beispiel_text)

    print(f"Anzahl der Chunks: {len(semantic_chunks)}")
    print(f"Parameter: threshold_type='{threshold_type}', threshold_amount={threshold_amount}\n")
    print("="*80)

    for i, chunk in enumerate(semantic_chunks, 1):
        print(f"\nChunk {i} ({len(chunk)} Zeichen):")
        print("-" * 80)
        print(chunk)
        print("-" * 80)

# Standard-Einstellungen
test_semantic_chunking()

### Beobachtung:

Die Chunks folgen thematischen Grenzen. Zum Beispiel bleiben alle Sätze über "Machine Learning" zusammen, auch wenn der Chunk größer wird.

## Übungen: Semantic Chunking verstehen und optimieren

### Aufgabe 1: Threshold-Type verstehen

Experimentiere mit verschiedenen `breakpoint_threshold_type` Optionen und beobachte, wie sich die Anzahl und Größe der Chunks verändert.

#### Experiment 1: Percentile mit niedrigem Wert (25. Perzentil)

Was passiert, wenn wir einen niedrigen Perzentil-Wert verwenden? Das bedeutet, wir trennen bereits bei relativ kleinen Ähnlichkeitsunterschieden.

In [ ]:
# Teste mit niedrigem Perzentil (mehr Chunks)
test_semantic_chunking(threshold_type="percentile", threshold_amount=25)

#### Experiment 2: Percentile mit hohem Wert (75. Perzentil)

Was passiert, wenn wir einen hohen Perzentil-Wert verwenden? Das bedeutet, wir trennen nur bei sehr großen Ähnlichkeitsunterschieden.

In [ ]:
# Teste mit hohem Perzentil (weniger Chunks)
test_semantic_chunking(threshold_type="percentile", threshold_amount=75)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

# Teile Text in Sätze auf (vereinfacht - in echt komplexer)
sentences = [s.strip() for s in beispiel_text.split('\n') if s.strip()]

# Erstelle Embeddings für alle Sätze
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
embeddings = embedding_model.embed_documents(sentences)

# Berechne Ähnlichkeiten zwischen aufeinanderfolgenden Sätzen
similarities = []
for i in range(len(embeddings) - 1):
    sim = cosine_similarity([embeddings[i]], [embeddings[i+1]])[0][0]
    similarities.append(sim)

# Visualisiere
plt.figure(figsize=(14, 6))
plt.plot(range(len(similarities)), similarities, marker='o', linewidth=2, markersize=8)
plt.axhline(y=np.percentile(similarities, 50), color='r', linestyle='--', label='50. Perzentil (Median)')
plt.axhline(y=np.percentile(similarities, 25), color='orange', linestyle='--', label='25. Perzentil')
plt.axhline(y=np.percentile(similarities, 75), color='green', linestyle='--', label='75. Perzentil')
plt.xlabel('Satz-Paar Index', fontsize=12)
plt.ylabel('Cosine Similarity', fontsize=12)
plt.title('Semantische Ähnlichkeit zwischen aufeinanderfolgenden Sätzen', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nStatistiken:")
print(f"Min Ähnlichkeit: {min(similarities):.3f}")
print(f"Max Ähnlichkeit: {max(similarities):.3f}")
print(f"Median (50. Perzentil): {np.percentile(similarities, 50):.3f}")
print(f"25. Perzentil: {np.percentile(similarities, 25):.3f}")
print(f"75. Perzentil: {np.percentile(similarities, 75):.3f}")

### Aufgabe 2: Visualisierung der semantischen Ähnlichkeiten

Um besser zu verstehen, wie Semantic Chunking funktioniert, können wir die Ähnlichkeiten zwischen aufeinanderfolgenden Sätzen visualisieren. Die **Täler** in der Grafik zeigen, wo das Chunking Grenzen setzt.

In [ ]:
# Teste mit hohem Perzentil (weniger Chunks)
test_semantic_chunking(threshold_type="percentile", threshold_amount=75)

## 3. LLM-basiertes semantisches Chunking

**Was ist der Unterschied zwischen embedding-basiertem und LLM-basiertem semantischem Chunking?**

Beide Methoden nutzen semantische Information, aber auf unterschiedliche Weise:

### Embedding-basiertes semantisches Chunking:
- **Wie:** Berechnet mathematische Vektoren (Embeddings) für Sätze und vergleicht deren Ähnlichkeit
- **Entscheidung:** Automatisch basierend auf Ähnlichkeitsschwellenwerten
- **Vorteile:** 
  - Schnell und deterministisch
  - Keine API-Kosten pro Chunking-Vorgang
  - Gut für große Textmengen
- **Nachteile:**
  - Nur mathematische Ähnlichkeit, kein tiefes Verständnis
  - Kann wichtige thematische Übergänge verpassen
  - Keine Berücksichtigung von Kontext oder Hierarchien

### LLM-basiertes semantisches Chunking:
- **Wie:** Ein Large Language Model analysiert den Text und identifiziert thematische Abschnitte
- **Entscheidung:** Intelligente Analyse durch das LLM basierend auf Sprach- und Kontextverständnis
- **Vorteile:**
  - Versteht Kontext, Hierarchien und implizite Zusammenhänge
  - Kann Überschriften/Themen generieren
  - Berücksichtigt Dokumentstruktur
- **Nachteile:**
  - Langsamer (API-Aufrufe)
  - Teurer (LLM-Nutzung)
  - Nicht deterministisch (kann bei gleichen Input leicht variieren)

**Wann welche Methode?**
- **Embedding-basiert**: Große Datenmengen, Geschwindigkeit wichtig, einfache Texte
- **LLM-basiert**: Komplexe Dokumente, Qualität wichtiger als Geschwindigkeit, strukturierte Ausgabe gewünscht

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# Vereinfachtes Beispiel - In der echten Implementierung komplexer
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Prompt für das LLM
prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein Experte für Textanalyse. Teile den folgenden Text in thematisch zusammenhängende Abschnitte auf."),
    ("user", """Analysiere diesen Text und teile ihn in 3-5 thematische Abschnitte auf.
    
Gib für jeden Abschnitt die Überschrift und den zugehörigen Text zurück.

Format:
ABSCHNITT 1: [Überschrift]
[Text des Abschnitts]

ABSCHNITT 2: [Überschrift]
[Text des Abschnitts]

...

Text:
{text}""")
])

chain = prompt | llm
result = chain.invoke({"text": beispiel_text})

print("LLM-basierte Chunking-Analyse:")
print("="*80)
print(result.content)

### Beobachtung:

Das LLM erkennt nicht nur thematische Grenzen, sondern kann auch:
- Hierarchien verstehen (z.B. "Deep Learning ist Teil von Machine Learning")
- Kontextabhängige Entscheidungen treffen
- Überschriften/Themen für Chunks generieren

## Aufgabe 3: Vergleich der Chunking-Strategien

Nachdem du die beiden semantischen Chunking-Methoden kennengelernt hast, möchtest du herausfinden, welche Strategie die bessere Retrieval-Qualität für dein RAG-System liefert: **Embedding-basiertes** oder **LLM-basiertes semantisches Chunking**.

### Schritt-für-Schritt Anleitung zur Evaluation

#### 1. Embedding-basiertes semantisches Chunking evaluieren

Öffne die `.env` Datei und setze:

```bash
CHUNKING_STRATEGY=SEMANTIC
SEMANTIC_BREAKPOINT_TYPE=percentile
SEMANTIC_BREAKPOINT_AMOUNT=50
```

#### 2. Backend neu starten

Damit die neuen Chunking-Parameter wirksam werden, muss das Backend neugestartet werden:

```bash
uv run --env-file .env python -m src.advanced_rag.backend.main
```

**Hinweis:** Das Backend muss laufen bleiben während der Evaluation!

#### 3. Evaluation-Dataset ausführen

Öffne ein neues Terminal (das Backend läuft im ersten Terminal) und führe aus:

```bash
uv run --env-file .env python src/advanced_rag/evaluation/evaluate_dataset.py
```

Dies führt alle Test-Queries gegen dein RAG-System aus und speichert die Ergebnisse in Langfuse.

#### 4. Ergebnisse in Langfuse notieren

1. Öffne das Langfuse Dashboard (URL aus deiner `.env` Datei: `LANGFUSE_HOST`)
2. Navigiere zum entsprechenden Projekt
3. Suche nach dem neuesten Evaluation-Run
4. Notiere die Metriken:
   - **Context Precision**: Wie relevant sind die abgerufenen Chunks?
   - **Context Recall**: Werden alle relevanten Informationen gefunden?
   - **Answer Relevancy**: Wie gut beantwortet das System die Frage?

#### 5. LLM-basiertes semantisches Chunking evaluieren

Wiederhole die Schritte 1-4 mit der LLM-basierten Strategie:

In der `.env` Datei setze:

```bash
CHUNKING_STRATEGY=SEMANTIC_LLM
```

Backend neu starten und Evaluation erneut durchführen.

#### 6. Ergebnisse vergleichen

**Embedding-basiertes semantisches Chunking:**
- Schnell und kostengünstig
- Basiert auf Ähnlichkeit zwischen Satz-Embeddings
- Gut für große Datenmengen

**LLM-basiertes semantisches Chunking:**
- Langsamer und teurer (nutzt LLM-API)
- Besseres Kontextverständnis
- Kann thematische Zusammenhänge und Hierarchien erkennen

**Notiere dir die Scores:**

| Strategie | Context Precision | Context Recall | Answer Relevancy |
|-----------|-------------------|----------------|------------------|
| Embedding-basiert  | ?                 | ?              | ?                |
| LLM-basiert | ?              | ?              | ?                |

**Frage zum Nachdenken:** Rechtfertigt die bessere Qualität des LLM-basierten Ansatzes die höheren Kosten und längere Verarbeitungszeit für deinen Use-Case?